In [1]:
import urllib.parse
import urllib.request
import json
import time
import pandas as pd
from sklearn.externals import joblib
import multiprocessing
import numpy as np

In [2]:
def getWeatherInfo():
    url = "http://api.openweathermap.org/data/2.5/weather?id=2964574&units=metric&APPID=31f19a108384bc317e2d91c5621c791e"
    with urllib.request.urlopen(url) as url:
        data = json.loads(url.read().decode('utf-8-sig'))
    # Return only the data for the next 24 hours
    weatherDict = {}
    weatherDict["temp"] = data["main"]["temp"]
    weatherDict["wind"] = data["wind"]["speed"]
    try:
        rain = data["main"]["rain"]
    except KeyError:
        rain = 0
    weatherDict["rain"] = rain
    return weatherDict

In [3]:
def timePredict(stopID, legRouteLine, prog_number, weekday, hour, route_start_stop, route_end_stop, weather):
    #    print()
    #    print("prog number ", prog_number)
    #    print()
    modelname = "models/" + legRouteLine + ".sav"
    normname = "models/n" + legRouteLine + ".sav"
    columnname = "models/c" + legRouteLine + ".csv"

    loaded_mlp = joblib.load(modelname)
    loaded_scaler = joblib.load(normname)
    columnsname = pd.read_csv(columnname).columns

    data = [{
        "stopID_" + str(stopID): 1,
        "progrnumber_" + str(prog_number): 1,
        "weekday_" + str(weekday): 1,
        "hour_" + str(hour): 1,
        "route_start_stop_" + str(route_start_stop): 1,
        "route_end_stop_" + str(route_end_stop): 1,
        "rain": weather['rain'],
        "temp": weather['temp'],
        "wdsp": weather['wind'],
    }]

    df = pd.DataFrame(data, columns=columnsname)
    #    print(df)
    df = df.fillna(0)
    normalized_df = loaded_scaler.transform(df)
    big_df = pd.DataFrame(normalized_df, columns=columnsname)
    X_df = big_df.drop(['target'], axis=1)
    target = loaded_mlp.predict(X_df)
    big_df['target'] = target
    predictionTime = loaded_scaler.inverse_transform(big_df)
    predictionTime = pd.DataFrame(predictionTime, columns=columnsname)
    predict = str(predictionTime['target']).replace(" ", "")
    predict = predict[1:]
    predict = predict.split(".")[0]
    #    print(predict)
    return int(predict)

In [4]:
weather = getWeatherInfo()

In [5]:
timetable = pd.read_csv("timetable.csv",dtype={"line_ID":str})
timetable.head()

,trip_id,departure_time,stop_headsign,prog_number,shape_dist_traveled,hour,distance,stop_id,line_ID,shape_id,weekday,previous_stopID,route_end_stop,route_start_stop,route_start_time,route_end_time
0,3643.y102m.60-1-d12-1.1.O,56400,Sandymount,1,0.000000,15,261.136188,226,1,60-1-d12-1.1.O,y102m,NaN,380,226,56400,59501
1,3643.y102m.60-1-d12-1.1.O,56447,Sandymount,2,261.136188,15,223.789101,228,1,60-1-d12-1.1.O,y102m,226.0,380,226,56400,59501
2,3643.y102m.60-1-d12-1.1.O,56486,Sandymount,3,484.925289,15,352.070390,229,1,60-1-d12-1.1.O,y102m,228.0,380,226,56400,59501
3,3643.y102m.60-1-d12-1.1.O,56550,Sandymount,4,836.995679,15,229.466104,227,1,60-1-d12-1.1.O,y102m,229.0,380,226,56400,59501
4,3643.y102m.60-1-d12-1.1.O,56589,Sandymount,5,1066.461783,15,292.654153,230,1,60-1-d12-1.1.O,y102m,227.0,380,226,56400,59501


In [17]:
len(timetable.line_ID.unique())

128

In [18]:
planned=[]
start_time = time.time()
for index, row in timetable.iterrows():
    if row["line_ID"]== "145":
        planned_departure_time =  timePredict(row["stop_id"],row["line_ID"],row["prog_number"],row["weekday"],row["hour"],row["route_start_stop"],row["route_end_stop"],weather)
        planned.append(planned_departure_time)
print((time.time() - start_time))

608.0721292495728


In [ ]:
# timetable["planned_departure_time"]=planned

# multiprocessing

In [6]:
timetable["planned_departure_time"] = np.nan

In [7]:
def func(d):
    for index, row in d.iterrows():
        if row["line_ID"] == "145":
            d.set_value(idx,"planned_departure_time",row["route_start_time"]+timePredict(row["stop_id"],row["line_ID"],row["prog_number"],row["weekday"],row["hour"],row["route_start_stop"],row["route_end_stop"],weather))
    return d

In [8]:
num_processes = multiprocessing.cpu_count()
chunk_size = int(timetable.shape[0]/num_processes)
chunks = [timetable.iloc[i:i + chunk_size,:] for i in range(0, timetable.shape[0], chunk_size)]

In [9]:
pool = multiprocessing.Pool(processes=num_processes)

In [ ]:
start_time = time.time()
result = pool.map(func, chunks)
print((time.time() - start_time))

In [ ]:
for i in range(len(result)):
   # since result[i] is just a dataframe
   # we can reassign the original dataframe based on the index of each chunk
   timetable.ix[result[i].index] = result[i]